# Example Podcast Workflow

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import subprocess
import os
import replicate
import soundfile as sf
from pydub import AudioSegment
import json
import feedparser
# custom library/module
import audio_utils as audio

## 1. Get a podcast episode from the RSS feed

One of the challenges is finding a podcast's RSS feed. I don't understand RSS
feeds, tbh. They seem to be pretty decentralized.

Here's an example podcast (Builders Build): https://anchor.fm/s/50fc9e0/podcast/rss

In [5]:
feed = feedparser.parse("https://anchor.fm/s/50fc9e0/podcast/rss")

In [37]:
# find the episode with this title ("What Exactly The Squad Does for a Living")
for entry in feed.entries:
    if "What Exactly The Squad Does for a Living" in entry.title:
        print(entry.title)
        print(entry.link)
        print(entry.published)
        print(entry.summary)
        print(entry.enclosures[0].href)
        print(entry.enclosures[0].type)
        print(entry.enclosures[0].length)
        #print(entry.enclosures[0].duration)
        url = entry.enclosures[0].href
        print('will use url={}'.format(url))

What Exactly The Squad Does for a Living, The Future of Accounting Services, Repurposing Content & Mew Talk
https://anchor.fm/buildersbuild/episodes/What-Exactly-The-Squad-Does-for-a-Living--The-Future-of-Accounting-Services--Repurposing-Content--Mew-Talk-e1mif9t
Tue, 16 Aug 2022 11:18:49 GMT
<p>Romans 12:2</p>
<p>Do not be conformed to this age, but be transformed by the renewing of your mind, so that you may discern what is the good, pleasing, and perfect will of God.</p>
<p>We had a good run of guests here on the Builders Build pod. We’re thankful for all of them, but it’s time for them to kick rocks. Somewhere along the way what we were putting out became too similar to what anyone can do, and those are the things that aren’t worth doing. Plus: we weren’t having much fun.</p>
<p>On this return to an open format with the gang, James, Oren, and Colin get right into it defining new terminology, answering the common “wtf do you guys do for a living?”, and finally getting into what they

In [38]:
# create a directory to store the audio files (res = resources)
os.makedirs('res', exist_ok=True)

In [39]:
# download the file and save it to the res directory with the name 'audio.m4a'
subprocess.call(['wget', url, '--directory-prefix', 'res/',
                 '--output-document', 'res/builders-build-audio.m4a'])

--2022-12-13 20:58:01--  https://anchor.fm/s/50fc9e0/podcast/play/56228605/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fstaging%2F2022-7-16%2F06e2f5c9-e22d-9452-8405-7818c36840cf.m4a
Resolving anchor.fm (anchor.fm)... 151.101.130.133, 151.101.2.133, 151.101.66.133, ...
Connecting to anchor.fm (anchor.fm)|151.101.130.133|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d3ctxlq1ktw2nl.cloudfront.net/staging/2022-7-16/06e2f5c9-e22d-9452-8405-7818c36840cf.m4a [following]
--2022-12-13 20:58:01--  https://d3ctxlq1ktw2nl.cloudfront.net/staging/2022-7-16/06e2f5c9-e22d-9452-8405-7818c36840cf.m4a
Resolving d3ctxlq1ktw2nl.cloudfront.net (d3ctxlq1ktw2nl.cloudfront.net)... 2600:9000:24eb:de00:9:8b5c:9c40:21, 2600:9000:24eb:3600:9:8b5c:9c40:21, 2600:9000:24eb:6400:9:8b5c:9c40:21, ...
Connecting to d3ctxlq1ktw2nl.cloudfront.net (d3ctxlq1ktw2nl.cloudfront.net)|2600:9000:24eb:de00:9:8b5c:9c40:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

0

## 2. Podcast file (m4a) -> .wav (necessary for Whisper to process)

In [42]:
audio.convert_m4a_to_wav('res/builders-build-audio.m4a',
                         'res/builders-build-audio.wav')

In [43]:
audio.calculate_audio_length('res/builders-build-audio.wav')

samples = 168755136
sample rate = 44100
seconds = 3826.647074829932
minutes = 63.77745124716554


# 3. get a 20 min clip (to save on Whisper API calls)

In [41]:
last_20_mins = 20 * 60 * 1000 # in milliseconds

In [44]:
clip = AudioSegment.from_file('res/builders-build-audio.wav', format='wav')
clip = clip[-last_20_mins:]
clip.export('res/builders-build-audio-20min.mp3', format='mp3')

<_io.BufferedRandom name='res/builders-build-audio-20min.mp3'>

In [46]:
audio.convert_mp3_to_wav('res/builders-build-audio-20min.mp3',
                         'res/builders-build-audio-20min.wav')

# 4. Process .wav file using Whisper -> Text Transcript

In [52]:
# use the .mp3 instead of .wav b/c it's smaller
audio_obj = open('res/builders-build-audio-20min.mp3', 'rb')

In [53]:
model = replicate.models.get('openai/whisper')
# this is the commit hash for the model (keeps changing)
version = model.versions.get("23241e5731b44fcb5de68da8ebddae1ad97c5094d24f94ccb11f7c1d33d661e2")
output = version.predict(audio=audio_obj, model='base')

In [54]:
audio.save_whisper_output('res/builders-build-audio-20min-output.json', output)

In [57]:
results = audio.read_whisper_json('res/builders-build-audio-20min-output.json')

In [60]:
# transcript has the 
results['transcription'][:500]

" There will always be something whatever comes next in just a way, but we haven't had that period for a couple of years since that kind of Really for more than a couple years since that first like Facebook ads bubble Let's say that was like got like seven years ago now, you know where you were really like Oh, we can make a lot of money by spending a little money This is this is a kind of a different one, but it requires you to make a lot of content I think James and I were we were all sharing in"

In [61]:
results['segments'][0]['text']

" There will always be something whatever comes next in just a way, but we haven't had that period for a couple of years since that kind of"